In [1]:
import pandas as pd
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense,GRU,Dropout
from keras.utils import to_categorical
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [3]:
# importing data
data = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/aashishksahu/Deep-Learning/master/data.csv")).dropna()
data.reindex(np.random.permutation(data.index))
print("dataset length: ",len(data))

dataset length:  19662


In [0]:
### preprocessing
# importing stopwords
from nltk.corpus import stopwords
stopwords = list(set(stopwords.words("english")))

# convert to list
review = list(data['Review Text'])[0:10000]

for i in range(len(review)):
    # remove unnecessary characters
    review[i] = re.findall(r'[a-zA-Z]\w+', review[i])
    # remove stopwords
    review[i] = [word for word in review[i] if word not in stopwords]
    # join words to form text
    review[i] = ' '.join(review[i])

# target 
recommendation = np.asarray(data['Recommended IND'])[0:10000]

In [0]:
# creating TF-IDF matrix
vectorizer = TfidfVectorizer()
vectorizer.fit(review)

# create tf-idf scores
tfidf = vectorizer.transform(review)
tfidf = tfidf.toarray()

# train-test split
x_train = tfidf[0:7000]
x_train = x_train.reshape(7000, 9679, 1)
y_train = recommendation[0:7000]

x_test = tfidf[7000:len(review)]
x_test = x_test.reshape(len(x_test), 9679, 1)
y_test = recommendation[7000:len(recommendation)]

# free memory
del(review)
del(recommendation)
del(data)
del(stopwords)

In [0]:
model = Sequential()
model.add(GRU(16, return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(GRU(8))
model.add(Dropout(0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss = "binary_crossentropy", optimizer="adam",  metrics = ['accuracy'])
model.fit(x_train, y_train.reshape(len(y_train),1), epochs=10, batch_size=512)

Epoch 1/10
6656/7000 [===========================>..] - ETA: 26s - loss: 2.8143 - acc: 0.8235 

In [0]:
# evaluation
model.evaluate(x_test, y_test.reshape(len(y_test),1))

In [0]:
# saving the trainied model
model.save("trained_model.h5")